# **Collaborative Filtering** 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme()
import warnings
warnings.filterwarnings('ignore')

## **Import datasets and feature engineering**

In [2]:
dataset = pd.read_csv("dataset.csv", sep = ',', index_col = 0)

dataset = dataset.drop_duplicates(keep = 'first')

dataset = dataset.drop(dataset[dataset['track_id'] == '1kR4gIb7nGxHPI3D2ifs59'].index)

dataset['artists'] = dataset['artists'].str.replace(';',', ')

dataset['artist_track'] = dataset[['artists', 'track_name']].agg(', '.join, axis = 1)

dataset = dataset.drop(columns = ['artists', 'album_name', 'track_name', 'duration_ms', 'explicit', 'track_id',
                                  'time_signature'])

In [3]:
dtypes = {'coordinates':str,
          'instrumentalness':float,
          'liveness':float,
          'speechiness':float,
          'danceability':float,
          'valence':float,
          'loudness':float,
          'tempo':float,
          'acousticness':float,
          'energy':float,
          'mode':int,
          'key':int,
          'artist_id':str,
          'place':str,
          'geo':str,
          'tweet_lang':str,
          'track_id':str,
          'created_at':object,
          'lang':str,
          'time_zone':str,
          'user_id':str,
          'id':str,
          'extra':str}

names = ['coordinates', 'instrumentalness', 'liveness', 'speechiness', 'danceability', 'valence', 'loudness', 
         'tempo', 'acousticness', 'energy', 'mode', 'key', 'artist_id', 'place', 'geo', 'tweet_lang', 'track_id', 
         'created_at', 'lang', 'time_zone', 'user_id', 'id', 'extra',]

ccf = pd.read_csv('context_content_features.csv', sep = ',', names = names, skiprows = 1)
ccf = ccf.drop(columns = ['coordinates', 'place', 'geo', 'tweet_lang', 'lang', 'time_zone', 'id', 'extra',
                         'artist_id'])
ccf = ccf.drop_duplicates()

#  

## **Create a dataset with common data**

In [4]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

df = dataset.merge(ccf, how = 'inner', on = ['instrumentalness','liveness','speechiness','danceability','valence', 
                                             'loudness','tempo','acousticness','energy','mode','key'])

df = df.drop(columns = ['key', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 
                        'instrumentalness', 'liveness', 'valence', 'tempo'])

df = df.drop_duplicates()
df = df.dropna()
df.head()

,popularity,track_genre,artist_track,track_id,created_at,user_id
0,68,acoustic,"Ron Pope, A Drop in the Ocean",531cef60f62729778e5c7cade58cca12,2014-01-03 19:42:53,124626736.00000
1,68,acoustic,"Ron Pope, A Drop in the Ocean",531cef60f62729778e5c7cade58cca12,2014-01-04 13:39:23,1340091181.00000
2,68,acoustic,"Ron Pope, A Drop in the Ocean",531cef60f62729778e5c7cade58cca12,2014-01-05 04:25:58,1313663000.00000
3,68,acoustic,"Ron Pope, A Drop in the Ocean",531cef60f62729778e5c7cade58cca12,2014-01-08 02:54:18,354961155.00000
4,68,acoustic,"Ron Pope, A Drop in the Ocean",531cef60f62729778e5c7cade58cca12,2014-01-08 15:29:21,225403852.00000


In [5]:
# Data Analysis 
print("Le dataset compte",len(df),"lignes au total, pour un nombre d'utilisateurs uniques de",
      df['user_id'].nunique(),".")

print("Le nombre moyen de morceaux écoutés par utilisateur est donc",
      np.round(len(df)/df['user_id'].nunique(), decimals = 0),".")

print("Ce jeu de données contient",df['artist_track'].nunique(),"morceaux uniques dans",
      df['track_genre'].nunique(),"genres différents.")

print("Le genre qui revient le plus souvent est le genre",df['track_genre'].mode()[0],"avec",
      df[df['track_genre'] == 'soul'].nunique()[2],"morceaux différents, présents",
      df['track_genre'].value_counts()[0],"fois dans notre jeu de données.")
print("Ces",df[df['track_genre'] == 'soul'].nunique()[2],"morceaux représentent donc à eux seuls",
      np.round(df['track_genre'].value_counts()[0]/len(df)*100, decimals = 0),"% des morceaux de notre dataset.")
print("La valeur moyenne de popularité pour l'ensemble des morceaux du jeu de données est",
      np.round(df['popularity'].mean(), decimals = 0),".")

Le dataset compte 64167 lignes au total, pour un nombre d'utilisateurs uniques de 10568 .
Le nombre moyen de morceaux écoutés par utilisateur est donc 6.0 .
Ce jeu de données contient 212 morceaux uniques dans 69 genres différents.
Le genre qui revient le plus souvent est le genre soul avec 2 morceaux différents, présents 23071 fois dans notre jeu de données.
Ces 2 morceaux représentent donc à eux seuls 36.0 % des morceaux de notre dataset.
La valeur moyenne de popularité pour l'ensemble des morceaux du jeu de données est 70.0 .


#  

## **Recommendation Function** 

In [6]:
def reco_col(track):
    users = list(df[df['artist_track'] == track]['user_id'].unique())
    
    genres = list(df[df['artist_track'] == track]['track_genre'].unique())

    playlist = df.loc[df['user_id'].isin(users)]
    playlist = playlist[playlist['artist_track'] != track]
    playlist = playlist.drop_duplicates(subset = ['artist_track'])
    playlist = playlist[~playlist['artist_track'].str.contains(track[:20])]
    playlist = playlist.sort_values('popularity', ascending = False)
    
    return playlist.loc[:, ["artist_track","track_genre","popularity"]]

track = "Carl Douglas, Kung Fu Fighting"
reco_col(track)

,artist_track,track_genre,popularity
37432,"John Legend, All of Me",soul,85
4292,"Gorillaz, Feel Good Inc.",alternative,84
4243,"Foo Fighters, Learn to Fly",alt-rock,77
61104,"Alejandro Sanz, Corazón partío",spanish,76
60592,"Jarabe De Palo, La flaca",spanish,76
...,...,...,...
29699,"Clannad, Theme from Harry's Game",new-age,21
29946,"Mike Oldfield, Ghost Bells",new-age,21
13243,"The Osborne Brothers, Bluegrass Concerto",bluegrass,20
63768,"Jon Kennedy, Funk Construction",trip-hop,19
